This version specifies a random portion of missingness in the features. It is identical with missing_cleanml-2-percentage. These two notebooks are used to run datasets in parallel. Airbnb in particular takes a long time to run.

In [1]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import utils
import pickle
from collections import defaultdict
import numpy as np
from collections import OrderedDict
from sklearn.metrics import f1_score 
import pandas as pd
from sklearn.metrics import f1_score 

/mnt/ds3lab-scratch/raox/gatech_data_cleaning/ICDE2021-revision/nacl36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/mnt/ds3lab-scratch/raox/gatech_data_cleaning/ICDE2021-revision/nacl36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import sys
sys.setrecursionlimit(10**6)

In [3]:
import math
import matplotlib.pyplot as plt 
from sklearn.datasets import load_digits
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pylab
import sys
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
import keras
import time
import pandas as pd
import json
import operator

Using TensorFlow backend.


In [4]:
sys.path.append("../NaCL/code/")
import utils
import LR2NB_GP
sys.path.append("../NaCL/notebooks/")
from dataload import *

In [5]:
from preprocess_mv import preprocess_data

/mnt/ds3lab-scratch/raox/gatech_data_cleaning/ICDE2021-revision/nacl36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [6]:
%load_ext autoreload
%autoreload 2

# Loading Data

In [7]:
seed = 3659

In [8]:
data_dir = "data-robustml-mv"

In [9]:
save_dir = "result-robustml"
model_name ='nacl'

In [10]:
accuracy = lambda x,y: np.mean(x == y)
f1 = lambda x,y: f1_score(x,y.flatten(), average = "weighted")



In [11]:
dataset = 'Marketing'

In [12]:
jobs = get_jobs_mv(data_dir, dataset = [dataset])
print("Number of jobs:", len(jobs))

Number of jobs: 20


In [13]:
jobs

[['data-robustml-mv', 'Marketing', 'missing_values', 'split_3462'],
 ['data-robustml-mv', 'Marketing', 'missing_values', 'split_3562'],
 ['data-robustml-mv', 'Marketing', 'missing_values', 'split_5056'],
 ['data-robustml-mv', 'Marketing', 'missing_values', 'split_5374'],
 ['data-robustml-mv', 'Marketing', 'missing_values', 'split_4225'],
 ['data-robustml-mv', 'Marketing', 'missing_values', 'split_144'],
 ['data-robustml-mv', 'Marketing', 'missing_values', 'split_7813'],
 ['data-robustml-mv', 'Marketing', 'missing_values', 'split_2516'],
 ['data-robustml-mv', 'Marketing', 'missing_values', 'split_905'],
 ['data-robustml-mv', 'Marketing', 'missing_values', 'split_2895'],
 ['data-robustml-mv', 'Marketing', 'missing_values', 'split_4764'],
 ['data-robustml-mv', 'Marketing', 'missing_values', 'split_7751'],
 ['data-robustml-mv', 'Marketing', 'missing_values', 'split_235'],
 ['data-robustml-mv', 'Marketing', 'missing_values', 'split_2962'],
 ['data-robustml-mv', 'Marketing', 'missing_values'

In [14]:
accuracy = lambda x,y: np.mean(x == y)
f1 = lambda x,y: f1_score(x,y.flatten(), average = "weighted")



In [15]:
with open('/mnt/ds3lab-scratch/raox/gatech_data_cleaning/ICDE2021-revision/CleanML/result/{}_result.json'.format(dataset)) as json_file:
    results = json.load(json_file)

In [17]:
%%time 

for job in jobs:
    
    print("Processing", job)
    print('--------------')
    dataset = job[1]

    dirty_X_train = pd.read_csv(os.path.join(*job, "dirty_X_train.csv"))
    dirty_y_train = pd.read_csv(os.path.join(*job, "dirty_y_train.csv"))
    dirty_X_test = pd.read_csv(os.path.join(*job, "dirty_X_test.csv"))
    dirty_y_test = pd.read_csv(os.path.join(*job, "dirty_y_test.csv"))

    dirty_X_train = dirty_X_train.fillna(0)
    dirty_X_test = dirty_X_test.fillna(0)
    X_train, y_train, X_test, y_test, mappings, mappings_dict = preprocess_data(dataset, 
                                                             dirty_X_train, dirty_y_train, 
                                                             [dirty_X_test], [dirty_y_test])
    X_test = X_test[0]
    y_test = y_test[0]
    dirty_X_test = pd.read_csv(os.path.join(*job, "dirty_X_test.csv"))
    null_data = dirty_X_test[dirty_X_test.isnull().any(axis=1)]

    missing = [[]] * dirty_X_test.shape[0]
    for i, row in null_data.iterrows():
        # append the original index of the missing feature
        cols = null_data.columns.get_values()[np.where(pd.isnull(row))]

        idx = [mappings_dict[col][0] for col in cols]
        missing[i] = idx
    missing = np.array(missing)

    output = defaultdict(lambda: defaultdict(list))

    output['train']['nacl'] = (X_train, y_train)
    output['test']['nacl'] = (X_test, y_test)
    output['missing_test'] = missing
    output['mappings1hot_dict'] = mappings_dict
    output['mappings1hot_list'] = mappings
    pickle.dump(dict(output), open(os.path.join(*job,"data.p"), 'wb'))


    max_acc = 0

    for method in ['delete', 'impute_holoclean', 'impute_mean_mode', 'impute_mean_dummy', 
                   'impute_median_mode', 'impute_median_dummy', 'impute_mode_mode', 'impute_mode_dummy']:
        #print('Titanic/v{}/missing_values/{}/logistic_regression/3659'.\
        #           format(split_seed, method))
        split_seed = job[-1].split("_")[1]
        a =  results['{}/v{}/missing_values/{}/logistic_regression/3659'.\
                   format(dataset, split_seed, method)].copy()
        #print(a)
        del a['best_params']
        del a['train_acc']
        del a['val_acc']
        max_key = max(a.items(), key=operator.itemgetter(1))[0]

        if a[max_key] > max_acc:
            best_params_lr = results['{}/v{}/missing_values/{}/logistic_regression/3659'.\
                   format(dataset, split_seed, method)]['best_params']
            max_acc = a[max_key]

    clf = LogisticRegression(solver='lbfgs', 
    verbose=True, 
    C = best_params_lr['C'],
    max_iter=2000, n_jobs=-1).fit(X_train, y_train)
    lr_pred = clf.predict(X_test)
    lr_train_acc = accuracy(clf.predict(X_train), y_train)
    lr_test_acc = accuracy(lr_pred, y_test)
    lr_test_f1 = f1(lr_pred, y_test)

    print("LR Train Accuracy: {} \nLR Test  Accuracy: {}".format(lr_train_acc, lr_test_acc))

    W = np.hstack((clf.intercept_[:,None], clf.coef_))[0]
    nacl = LR2NB_GP.LR2NB(W)
    nacl.setObj(X_train, y_train)

    # Using Mosek Solver
    nacl.solve(verbose=1)

    # Sanity Check to compare NaCL results with Logistic Regression
    assert(1.0 == np.average( nacl.classify(X_test)  == clf.predict(X_test )))
    assert(1.0 == np.average( nacl.classify(X_train) == clf.predict(X_train)))
    nacl_pred = nacl.classify(X_test)
    nacl_train_acc = accuracy(nacl.classify(X_train), y_train)
    nacl_test_acc = accuracy(nacl_pred, y_test)
    #print(nacl_pred)
    #print(y_test)
    nacl_test_f1 = f1(nacl_pred[0], y_test)

    print("NaCL Train Accuracy: {} \nNaCL Test  Accuracy: {}".format(nacl_train_acc, nacl_test_acc))

    NB = BernoulliNB().fit(X_train, y_train)
    NB_pred = NB.predict(X_test)
    nb_train_acc = accuracy(NB.predict(X_train), y_train)
    nb_test_acc = accuracy(NB_pred, y_test)
    nb_test_f1 = f1(NB_pred, y_test)

    print("NB Train Accuracy: {} \nNB Test  Accuracy: {}".format(nb_train_acc, nb_test_acc))

    # Make an example observations with some missing features

    # setting the dimensions to be zero
    missing_mask = np.zeros(X_test.shape) 


    miss = missing
    for i in range(len(missing_mask)):
        for j in range(len(miss[i])):
            missing_mask[i][mappings[miss[i][j]]] = 1

    a = nacl.predict_proba(X_test, missing = missing_mask)[:,1]
    a = np.where(a > 0.5, 1, 0)

    test_acc = accuracy(y_test, a)
    test_f1 = f1(y_test, a)
    print('nacl test acc with missing:', test_acc)
    print('nacl test f1 with missing:', test_f1) # true, pred


    result_dict = {}

    result_dict["best_params"] = best_params_lr


    result_dict["{}_test_f1".format('nacl_missing')] = test_f1
    result_dict["{}_test_acc".format('nacl_missing')] = test_acc

    result_dict["{}_test_f1".format('nacl')] = nacl_test_f1
    result_dict["{}_test_acc".format('nacl')] = nacl_test_acc
    result_dict["{}_train_acc".format('nacl')] = nacl_train_acc

    result_dict["{}_test_f1".format('nb')] = nb_test_f1
    result_dict["{}_test_acc".format('nb')] = nb_test_acc
    result_dict["{}_train_acc".format('nb')] = nb_train_acc

    result_dict["{}_test_f1".format('lr')] = lr_test_f1
    result_dict["{}_test_acc".format('lr')] = lr_test_acc
    result_dict["{}_train_acc".format('lr')] = lr_train_acc

    print(result_dict)
    save_result_mv(job, seed, result_dict, model_name, save_dir)





Processing ['data-robustml-mv', 'Marketing', 'missing_values', 'split_3462']
--------------
dataset: {'data_dir': 'Marketing', 'error_types': ['missing_values'], 'label': 'Income', 'ml_task': 'classification'}
columns to encode: ['Sex', 'Marital', 'Age', 'Education', 'Occupation', 'Live', 'Dual', 'Person', 'Person under 18', 'Householder', 'Hometype', 'Ethnic', 'Language']


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.7s finished


LR Train Accuracy: 0.775412960609911 
LR Test  Accuracy: 0.7719688542825361
Using solver 'cvxopt'
 for 338 free variables
  in 340 posynomial inequalities.
Solving took 1.35 seconds.
NaCL Train Accuracy: 0.775412960609911 
NaCL Test  Accuracy: 0.7719688542825361
NB Train Accuracy: 0.7577827191867853 
NB Test  Accuracy: 0.7675194660734149
nacl test acc with missing: 0.7693733778272154
nacl test f1 with missing: 0.7685629215786406
{'best_params': {'C': 0.004321886239791225}, 'nacl_missing_test_f1': 0.7685629215786406, 'nacl_missing_test_acc': 0.7693733778272154, 'nacl_test_f1': 0.7723230126062666, 'nacl_test_acc': 0.7719688542825361, 'nacl_train_acc': 0.775412960609911, 'nb_test_f1': 0.7670942721995886, 'nb_test_acc': 0.7675194660734149, 'nb_train_acc': 0.7577827191867853, 'lr_test_f1': 0.7723230126062666, 'lr_test_acc': 0.7719688542825361, 'lr_train_acc': 0.775412960609911}
Processing ['data-robustml-mv', 'Marketing', 'missing_values', 'split_3562']
--------------
dataset: {'data_dir': 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.1s finished


LR Train Accuracy: 0.7760482846251588 
LR Test  Accuracy: 0.7730812013348165
Using solver 'cvxopt'
 for 338 free variables
  in 340 posynomial inequalities.
Solving took 3.78 seconds.
NaCL Train Accuracy: 0.7760482846251588 
NaCL Test  Accuracy: 0.7730812013348165
NB Train Accuracy: 0.7565120711562897 
NB Test  Accuracy: 0.7615869484612533
nacl test acc with missing: 0.7715980719317761
nacl test f1 with missing: 0.7712303387554902
{'best_params': {'C': 0.5265071321453342}, 'nacl_missing_test_f1': 0.7712303387554902, 'nacl_missing_test_acc': 0.7715980719317761, 'nacl_test_f1': 0.7732225025858452, 'nacl_test_acc': 0.7730812013348165, 'nacl_train_acc': 0.7760482846251588, 'nb_test_f1': 0.7610715471467953, 'nb_test_acc': 0.7615869484612533, 'nb_train_acc': 0.7565120711562897, 'lr_test_f1': 0.7732225025858452, 'lr_test_acc': 0.7730812013348165, 'lr_train_acc': 0.7760482846251588}
Processing ['data-robustml-mv', 'Marketing', 'missing_values', 'split_5056']
--------------
dataset: {'data_dir'

...

LR Train Accuracy: 0.7744599745870394 
LR Test  Accuracy: 0.7793845012977382
Using solver 'cvxopt'
 for 338 free variables
  in 340 posynomial inequalities.
Solving took 5.39 seconds.
NaCL Train Accuracy: 0.7744599745870394 
NaCL Test  Accuracy: 0.7793845012977382
NB Train Accuracy: 0.7571473951715375 
NB Test  Accuracy: 0.7582499073044123
nacl test acc with missing: 0.778642936596218
nacl test f1 with missing: 0.7785592099555718
{'best_params': {'C': 0.009878597119522799}, 'nacl_missing_test_f1': 0.7785592099555718, 'nacl_missing_test_acc': 0.778642936596218, 'nacl_test_f1': 0.7793630480572522, 'nacl_test_acc': 0.7793845012977382, 'nacl_train_acc': 0.7744599745870394, 'nb_test_f1': 0.7578052578792318, 'nb_test_acc': 0.7582499073044123, 'nb_train_acc': 0.7571473951715375, 'lr_test_f1': 0.7793630480572522, 'lr_test_acc': 0.7793845012977382, 'lr_train_acc': 0.7744599745870394}
Processing ['data-robustml-mv', 'Marketing', 'missing_values', 'split_6542']
--------------
dataset: {'data_dir'

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.1s finished


LR Train Accuracy: 0.7803367217280813 
LR Test  Accuracy: 0.7727104189840563
Using solver 'cvxopt'
 for 338 free variables
  in 340 posynomial inequalities.
Solving took 5.89 seconds.
NaCL Train Accuracy: 0.7803367217280813 
NaCL Test  Accuracy: 0.7727104189840563
NB Train Accuracy: 0.7609593392630242 
NB Test  Accuracy: 0.7515758249907304
nacl test acc with missing: 0.7697441601779755
nacl test f1 with missing: 0.7698091724552233
{'best_params': {'C': 0.576379223702522}, 'nacl_missing_test_f1': 0.7698091724552233, 'nacl_missing_test_acc': 0.7697441601779755, 'nacl_test_f1': 0.7724355556067314, 'nacl_test_acc': 0.7727104189840563, 'nacl_train_acc': 0.7803367217280813, 'nb_test_f1': 0.750875248504193, 'nb_test_acc': 0.7515758249907304, 'nb_train_acc': 0.7609593392630242, 'lr_test_f1': 0.7724355556067314, 'lr_test_acc': 0.7727104189840563, 'lr_train_acc': 0.7803367217280813}
Processing ['data-robustml-mv', 'Marketing', 'missing_values', 'split_8444']
--------------
dataset: {'data_dir': 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.1s finished


LR Train Accuracy: 0.7752541296060991 
LR Test  Accuracy: 0.7734519836855765
Using solver 'cvxopt'
 for 338 free variables
  in 340 posynomial inequalities.
Solving took 3.3 seconds.
NaCL Train Accuracy: 0.7752541296060991 
NaCL Test  Accuracy: 0.7734519836855765
NB Train Accuracy: 0.7566709021601017 
NB Test  Accuracy: 0.7667779013718947
nacl test acc with missing: 0.7730812013348165
nacl test f1 with missing: 0.7731307746932726
{'best_params': {'C': 0.0029327085198583313}, 'nacl_missing_test_f1': 0.7731307746932726, 'nacl_missing_test_acc': 0.7730812013348165, 'nacl_test_f1': 0.7732039545768062, 'nacl_test_acc': 0.7734519836855765, 'nacl_train_acc': 0.7752541296060991, 'nb_test_f1': 0.7659273927606595, 'nb_test_acc': 0.7667779013718947, 'nb_train_acc': 0.7566709021601017, 'lr_test_f1': 0.7732039545768062, 'lr_test_acc': 0.7734519836855765, 'lr_train_acc': 0.7752541296060991}
CPU times: user 2min 29s, sys: 7min 17s, total: 9min 46s
Wall time: 2min 22s
